# Kaggle (Kyivstar)

In [4]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from numpy.random import RandomState
import xgboost as xgb
import copy 

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.grid_search import ParameterGrid
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.cluster import KMeans

import sklearn.metrics
from sklearn.utils import shuffle
from sklearn import cross_validation as cv
 # Feature Importance
from sklearn import datasets
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier#plt.hist(data_train[['VOICE_SCH_MORN_6M']])
#plt.show()

### load data

In [43]:
data_train = pd.read_csv(r'A:\KAGGLE\Kaggle_Kyivstar\Xtrain.csv', sep = ',')

In [44]:
print(data_train.shape)

(8532, 42)


### build a single graph

In [7]:
fig, ax = plt.subplots()
ax.scatter(data_train[['IS_RETIREE']], data_train[['REFILLED_3M']])
plt.show()

### count and investigate values of SUPPORT_3G as it consists of only NAs and 1es

In [45]:
print('is retiree and supports 3G: ',sum(data_train['IS_RETIREE'][data_train['SUPPORT_3G']==1]))
print('supports 3G and is not retiree: ',data_train['SUPPORT_3G'][data_train['IS_RETIREE']==0].sum(skipna=True))

is retiree and supports 3G:  192
supports 3G and is not retiree:  2647.0


In [46]:
print(data_train['HANDSET_TYPE'].unique().shape)
print(data_train['AVG_CHARGE_3M_GROUP'].unique().shape)

(7,)
(5,)


### create new categorical and binary variables

In [47]:
category = list(set(data_train['HANDSET_VENDOR_CVAL']))
category
data_train['HANDSET_VENDOR_CVAL_ix'] = [category.index(i) for i in data_train['HANDSET_VENDOR_CVAL']]

category = list(set(data_train['HANDSET_TYPE']))
category
data_train['HANDSET_TYPE_ix'] = [category.index(i) for i in data_train['HANDSET_TYPE']]
data_train['AVG_CHARGE_3M_GROUP'] = data_train['AVG_CHARGE_3M_GROUP']-1
#data_train['HANDSET_TYPE_ix_grouped'] = [[category.index(i),1][category.index(i) in [2,3,4,6]] for i in data_train['HANDSET_TYPE']]
data_train = pd.concat([data_train, pd.get_dummies(data_train['AVG_CHARGE_3M_GROUP'],prefix='gr')], axis=1)
data_train = pd.concat([data_train, pd.get_dummies(data_train['HANDSET_TYPE_ix'],prefix='handset')], axis=1)

## Correlation:
dd = data_train.corr()
## Features without dummies
all_features1 = [i for i in list(data_train.columns.values) if i not in ['SUBS_ID','HANDSET_VENDOR_CVAL','HANDSET_TYPE', 
    'gr_1','gr_2','gr_3','gr_4','gr_0','handset_0','handset_1','handset_2','handset_3','handset_4','handset_5','handset_6']]
## Features with dummies                                                                    
all_features2 = [i for i in list(data_train.columns.values) if i not in ['SUBS_ID','HANDSET_VENDOR_CVAL','HANDSET_TYPE', 
    'AVG_CHARGE_3M_GROUP','HANDSET_TYPE_ix','gr_0','handset_0']] 

print(len(all_features1), len(all_features2))

41 49


In [48]:
## Show all pairs of highly correlated classes:
s = list()
for i in range(len(all_features1)):
    for j in range(i,len(all_features1)):
        f1 = all_features1[i]
        f2 = all_features1[j]
        if dd[f1][f2]>0.9 and f1!=f2:
            s.append([f1,f2])
for pair in s:
    print(pair)        

['V_OUT_CNT_3M', 'VOICE_CNT_WRK_DAY_6M']
['DATA_MB_3M', 'DATA_CNT_EV_6M']
['DATA_MB_3M', 'DATA_CNT_WRK_AFT_EV_6M']
['DATA_MB_3M', 'DATA_CNT_WRK_DAY_6M']
['OUT_ROUM_MIN_3M', 'OUT_ROUM_CNT_3M']
['OUT_ROUM_MIN_3M', 'IN_ROUM_CNT_3M']
['IN_ROUM_MIN_3M', 'IN_ROUM_CNT_3M']
['OUT_ROUM_CNT_3M', 'IN_ROUM_CNT_3M']
['DATA_CNT_EV_6M', 'DATA_CNT_WRK_AFT_EV_6M']
['DATA_CNT_SCH_MORN_6M', 'DATA_CNT_WRK_DAY_6M']
['SMS_CNT_EV_6M', 'SMS_CNT_NGHT_6M']
['SMS_CNT_EV_6M', 'SMS_CNT_WRK_AFT_EV_6M']
['SMS_CNT_EV_6M', 'SMS_CNT_WRK_DAY_6M']
['SMS_CNT_NGHT_6M', 'SMS_CNT_WRK_AFT_EV_6M']
['SMS_CNT_NGHT_6M', 'SMS_CNT_WRK_DAY_6M']
['SMS_CNT_WRK_AFT_EV_6M', 'SMS_CNT_WRK_DAY_6M']
['VOICE_CNT_EV_6M', 'VOICE_WRK_AFT_EV_6M']
['VOICE_SCH_MORN_6M', 'VOICE_CNT_WRK_DAY_6M']
['VOICE_DUR_EV_6M', 'VOICE_DUR_WRK_AFT_EV_6M']


In [49]:
## summarize features:
for col in all_features1:
    print(col," mean:", data_train[col].mean(), " median:",data_train[col].median(), " max:",data_train[col].max(), 
          " min:",data_train[col].min(), " count_non_na:",data_train[col].count(),
         #" cat:",pd.Categorical(data_train[col]).dtype.name,
         " count:",len(pd.Series.unique(data_train[col])))
    #data_train[col] = data_train[col].fillna(0)

AVG_CHARGE_3M_GROUP  mean: 2.0042194092827006  median: 2.0  max: 4  min: 0  count_non_na: 8532  count: 5
V_OUT_INT_DUR_3M  mean: 0.002877974446137616  median: 0.0  max: 1.0  min: 0.0  count_non_na: 8531  count: 133
V_IN_INT_DUR_3M  mean: 0.0015301840346969807  median: 0.0  max: 1.0  min: 0.0  count_non_na: 8531  count: 81
V_OUT_INT_CNT_3M  mean: 0.004047122259992959  median: 0.0  max: 1.0  min: 0.0  count_non_na: 8531  count: 123
V_IN_INT_CNT_3M  mean: 0.006025905521040979  median: 0.0  max: 1.0  min: 0.0  count_non_na: 8531  count: 142
V_OUT_DUR_3M  mean: 0.04256417770484064  median: 0.022000000000000002  max: 1.0  min: 0.0  count_non_na: 8531  count: 322
V_IN_DUR_3M  mean: 0.056199976556088896  median: 0.037000000000000005  max: 1.0  min: 0.0  count_non_na: 8531  count: 350
V_OUT_CNT_3M  mean: 0.08585464775524472  median: 0.055  max: 1.0  min: 0.0  count_non_na: 8531  count: 466
V_IN_CNT_3M  mean: 0.07334415660532131  median: 0.052000000000000005  max: 1.0  min: 0.0  count_non_na: 85

### create new dataset with dropped NA values, and dataset with NAs filled by zeros (medians)

In [50]:
features1 = [i for i in all_features1 if i not in ['IS_RETIREE',]]
features2 = [i for i in all_features2 if i not in ['IS_RETIREE',]]
print(len(data_train[features1]), len(data_train[features1].dropna()))
data_train['SUPPORT_3G'] = data_train['SUPPORT_3G'].fillna(0)

train_na_dropped = data_train.dropna()
data_train_with_na = copy.deepcopy(data_train)

for f in data_train.columns.values:
    data_train[f] = data_train[f].fillna(0)
rng = sklearn.utils.check_random_state(0)
print("data_train shape is:", data_train.shape, " train_na_dropped shape is:", train_na_dropped.shape)
#x_train, y_train = shuffle(data_train[features], data_train['IS_RETIREE'], random_state=0)

8532 2803
data_train shape is: (8532, 56)  train_na_dropped shape is: (8384, 56)


## Initial check of all classifiers for different number of k best features

In [30]:
def choose_k_best(k,dataset,features):
    selection = SelectKBest(k=k)
    return SelectKBest(k=k).fit_transform(dataset[features],dataset['IS_RETIREE'])

def init_Check(features, dataset, k_list):  
    def score_model(model):
        return cv.cross_val_score(model, X, y, cv=skf, scoring=score_metric)
    for k in k_list:
        data_kbest = choose_k_best(k, dataset, features)   
        X, y = shuffle(data_kbest, dataset['IS_RETIREE'], random_state=0)
        
        skf = cv.StratifiedKFold(y, n_folds=10, shuffle=True)
        score_metric = 'roc_auc'
        scores = {}
        
        scores['svc'] = score_model(SVC())
        scores['tree'] = score_model(DecisionTreeClassifier()) 
        scores['extra_tree'] = score_model(ExtraTreesClassifier())
        scores['forest'] = score_model(RandomForestClassifier())
        scores['ada_boost'] = score_model(AdaBoostClassifier())
        scores['bagging-tree'] = score_model(BaggingClassifier())
        scores['bagging-forest'] = score_model(BaggingClassifier(base_estimator = RandomForestClassifier()))
        scores['bagging-ridge'] = score_model(BaggingClassifier(base_estimator = RidgeClassifier()))
        scores['grad_boost'] = score_model(GradientBoostingClassifier())
        scores['ridge'] = score_model(RidgeClassifier())
        scores['passive'] = score_model(PassiveAggressiveClassifier())
        scores['sgd'] = score_model(SGDClassifier())
        scores['gaussian'] = score_model(GaussianNB())
        scores['xgboost'] = score_model(XGBClassifier())

        # Print the scores
        model_scores = pd.DataFrame(scores).mean()
        model_scores.sort_values(ascending=False)
        model_scores.to_csv('model_scores.csv', index=False)
        print("Number of best features chosen:",k)
        print('Model scores\n{}'.format(model_scores))

print("\nWithout dummies, NAs filled with 0:")
k_list = [25, 35, 38, len(features1)]
init_Check(features1, data_train, k_list)
print("\nWithout dummies, NAs dropped:")
init_Check(features1, train_na_dropped, k_list)

print("\nWith dummies, NAs filled with 0:")
k_list = [35, 45, 48, len(features2)]
init_Check(features2, data_train, k_list)
print("\nWith dummies, NAs dropped:")
init_Check(features2, train_na_dropped, k_list)

Without dummies, NAs filled with 0:

Number of best features chosen: 25
Model scores
ada_boost         0.741363
bagging-forest    0.711883
bagging-ridge     0.725037
bagging-tree      0.643377
extra_tree        0.652038
forest            0.643786
gaussian          0.695699
grad_boost        0.749647
passive           0.604232
ridge             0.724796
sgd               0.614815
svc               0.602776
tree              0.548438
xgboost           0.754451
dtype: float64
Number of best features chosen: 35
Model scores
ada_boost         0.741043
bagging-forest    0.717574
bagging-ridge     0.724555
bagging-tree      0.656849
extra_tree        0.641960
forest            0.656977
gaussian          0.684960
grad_boost        0.749444
passive           0.613302
ridge             0.724591
sgd               0.615896
svc               0.630484
tree              0.550261
xgboost           0.751782
dtype: float64
Number of best features chosen: 38
Model scores
ada_boost         0.736994
baggin

### Bagging classificator with built in cross-validation

In [14]:
def k_fold_cross_validation_bagging(k, x_data, y_data, base_estimator, rng, params):
    n = len(x_data)
    AUC_list = list()
    for i in range(k):
        start = int((n*i)/k)
        end = int((n*(i+1))/k-1)
        x_valid = x_data[start:end+1]
        y_valid = y_data[start:end+1] 
        x_train = x_data[:start].append(x_data[end+1:])
        y_train = y_data[:start].append(y_data[end+1:])
        
        predictions = BaggingClassifier(base_estimator = base_estimator, n_estimators = 40,
                        random_state = rng,
                        **params).fit(x_train, y_train).predict_proba(x_valid)
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_valid, predictions[:,1], pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        AUC_list.append(auc)
    return (np.mean(AUC_list))

In [53]:
x_train, y_train = shuffle(data_train[features2], data_train['IS_RETIREE'], random_state=0)

grid = ParameterGrid({"max_samples" : [0.5, 0.75],
                      "max_features": [0.5,],
                      "bootstrap": [False,],
                      "bootstrap_features": [False,] })
base_estimators = [
    RandomForestClassifier(n_estimators=35),
    RidgeClassifier(),
                   
                  ]
est_names = ['FOREST_35',
             'RIDGE',
            ]
i = 0
for base_estimator in base_estimators:
    for params in grid:
        auc = k_fold_cross_validation_bagging(10, x_train, y_train, base_estimator, rng, params)
        print("bootstrap:",params['bootstrap'], "  max_samples:", params['max_samples'],"  max_features:", params['max_features'], 
              "  AUC:",auc, "  estimator:",est_names[i],"   bootstrap_features:",params["bootstrap_features"])
    i += 1    


bootstrap: False   max_samples: 0.5   max_features: 0.5   AUC: 0.741495039464   estimator: FOREST_35    bootstrap_features: False
bootstrap: False   max_samples: 0.75   max_features: 0.5   AUC: 0.744104279611   estimator: FOREST_35    bootstrap_features: False
bootstrap: False   max_samples: 0.5   max_features: 0.5   AUC: 0.500716294807   estimator: RIDGE    bootstrap_features: False
bootstrap: False   max_samples: 0.75   max_features: 0.5   AUC: 0.500782125858   estimator: RIDGE    bootstrap_features: False


## PCA and K-means datasets

In [15]:
train_st = StandardScaler().fit_transform(data_train[features1])
x_train, y_train = shuffle(train_st, data_train['IS_RETIREE'], random_state=rng)
pca = PCA(n_components=30)
# Maybe some original features where good, too?
selection = SelectKBest(k=1)
# Build estimator from PCA and Univariate selection:
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
# Use combined features to transform dataset:
cf = combined_features.fit(x_train, y_train)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
X_features = cf.transform(x_train)

train_pca = pd.DataFrame(X_features)
#print(train_pca[:5])
def create_data_kmeans(k, data_train, features):
    data_kmeans = (KMeans(n_clusters = k)).fit(data_train[features],data_train['IS_RETIREE'])
    data_kmeans = (data_kmeans.transform(data_train[features],data_train['IS_RETIREE']))
    print(data_kmeans.shape)
    return data_kmeans
X_features = create_data_kmeans(25, data_train, features1) 
train_kmeans = pd.DataFrame(X_features)

[ 0.19872057  0.13157198  0.11285504  0.0985253   0.09003118  0.05720297
  0.03867009  0.03659307  0.02976893  0.02387298  0.02071065  0.01862513
  0.01826621  0.01600806  0.01173828  0.01124685  0.01012342  0.0097324
  0.00896957  0.00832734  0.00772957  0.00703826  0.00633228  0.00469435
  0.00442665  0.0037168   0.00257133  0.00224181  0.00212544  0.00153674]
0.993973264886
(8532, 25)


## ...

In [55]:
print(train_kmeans.shape, train_pca.shape)

(8532, 15) (8532, 28)


## XGBoost with built in cross-validation

In [61]:
def k_fold_cross_validation_xgboost(k, x_data, y_data, params, features):
    n = len(x_data)
    AUC_list = list()
    #imp = np.array([0]*len(features))
    for i in range(k):
        start = int((n*i)/k)
        end = int((n*(i+1))/k-1)
        x_valid = x_data[start:end+1]
        y_valid = y_data[start:end+1] 
        x_train = x_data[:start].append(x_data[end+1:])
        y_train = y_data[:start].append(y_data[end+1:])
        
        clf = xgb.XGBClassifier(**params)
        x_fit, x_eval, y_fit, y_eval= train_test_split(x_train, y_train, test_size=0.2)
        clf = clf.fit(x_fit, y_fit, 
                              early_stopping_rounds=30, 
                              eval_metric="auc", 
                eval_set=[(x_eval, y_eval)])
        predictions = clf.predict_proba(x_valid)
        #imp = imp + np.array(clf.feature_importances_)                  
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_valid, predictions[:,1], pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        AUC_list.append(auc)
    #print(sorted(dict(zip(features,imp)).items(), key = lambda x: x[1]))    
    return (np.mean(AUC_list))

In [80]:
grid_xgb = ParameterGrid({
                      "gamma":[0,],
                      "missing" : [np.nan,],
                      "max_depth": [5,],
                      "n_estimators": [400,],
                      "learning_rate": [0.03,],
                      "nthread":[5,],
                      "subsample":[0.8,],
                      "colsample_bytree":[0.85,],
                      "seed":[4242,]})

## initial dataset with NAs:
x_train, y_train = shuffle(data_train_with_na[features1], data_train_with_na['IS_RETIREE'], random_state=rng)
for params in grid_xgb:
    print(" n_est:",params['n_estimators']," subsample:",params['subsample']," nthread:",params['nthread'],
        " colsample_bytree:",params['colsample_bytree'], " max_depth:",params['max_depth'],
        " AUC:",k_fold_cross_validation_xgboost(10, x_train, y_train, params, features1))

## distances to clusters(kmeans):
x_train, y_train = shuffle(train_kmeans, data_train['IS_RETIREE'], random_state=rng)
for params in grid_xgb:
    print(" n_est:",params['n_estimators']," subsample:",params['subsample']," nthread:",params['nthread'],
        " colsample_bytree:",params['colsample_bytree'], " max_depth:",params['max_depth'],
        " AUC:",k_fold_cross_validation_xgboost(10, x_train, y_train, params, features1))

## PCA dataset:    
x_train, y_train = shuffle(train_pca, data_train['IS_RETIREE'], random_state=rng)
for params in grid_xgb:
    print(" n_est:",params['n_estimators']," subsample:",params['subsample']," nthread:",params['nthread'],
        " colsample_bytree:",params['colsample_bytree'], " max_depth:",params['max_depth'],
        " AUC:",k_fold_cross_validation_xgboost(10, x_train, y_train, params, features1))
        

 n_est: 400  subsample: 0.8  nthread: 5  colsample_bytree: 0.85  max_depth: 5  AUC: 0.751344714123
 n_est: 400  subsample: 0.8  nthread: 5  colsample_bytree: 0.85  max_depth: 5  AUC: 0.659279856614
 n_est: 400  subsample: 0.8  nthread: 5  colsample_bytree: 0.85  max_depth: 5  AUC: 0.505545359945


## check for other classifier with all features

In [143]:
def k_fold_cross_validation_class(k, classifier, x_data, y_data, features):
    n = len(x_data)
    AUC_list = list()
    imp = np.array([0]*len(features))
    for i in range(k):
        start = int((n*i)/k)
        end = int((n*(i+1))/k-1)
        x_valid = x_data[start:end+1]
        y_valid = y_data[start:end+1] 
        x_train = x_data[:start].append(x_data[end+1:])
        y_train = y_data[:start].append(y_data[end+1:])
        
        predictions = classifier.fit(x_train, y_train).predict_proba(x_valid)
        imp = imp + np.array(classifier.feature_importances_)
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_valid, predictions[:,1], pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        AUC_list.append(auc)
    print(sorted(dict(zip(features,imp)).items(), key = lambda x: x[1])[:5])
    return np.mean(AUC_list)

In [147]:
x_train, y_train = shuffle(data_train[features1], data_train['IS_RETIREE'], random_state=rng)
#x_train, y_train = shuffle(x_train_pca, train['IS_RETIREE'], random_state=0)

grid1 = ParameterGrid({"n_estimators" : [360, ],
                      "learning_rate": [0.1,],
                      "subsample": [1,],
                      "max_leaf_nodes": [4,] ,
                     "min_samples_split":[5,],
                     "max_features": [1,],
                     "max_depth": [None,],
                     "random_state":[0,]}) 
for params in grid1:
    classifier = GradientBoostingClassifier(**params)
    print("GrBoost_AUC:",k_fold_cross_validation_class(10,classifier, x_train, y_train, features1),
         " learning_rate:",params['learning_rate']," n_est:",params['n_estimators'],
         " max_leaf_nodes",params['max_leaf_nodes'], " min_sample_split:",params['min_samples_split'],
         " max_features",params['max_features'],'\n\n') 

grid2 = ParameterGrid({"n_estimators" : [300,],"learning_rate": [0.1,],"random_state":[0,]}) 
params0 = grid1[0]
base_estimator = GradientBoostingClassifier(**params0)
for params in grid2:
    classifier = AdaBoostClassifier(base_estimator = base_estimator,**params)
    print("AdaBoost_AUC:",k_fold_cross_validation_class(10,classifier, x_train, y_train, features1),
         " learning_rate:",params['learning_rate']," n_est:",params['n_estimators'],'\n\n')


[('OUT_ROUM_MIN_3M', 0.059629811231823746), ('OUT_ROUM_CNT_3M', 0.061066518814388709), ('IN_ROUM_CNT_3M', 0.064297190797614992), ('DATA_CNT_SCH_MORN_6M', 0.070204144191630916), ('SMS_CNT_EV_6M', 0.070684634117065687)]
GrBoost_AUC: 0.752517522121  learning_rate: 0.1  n_est: 360  max_leaf_nodes 4  min_sample_split: 5  max_features 1 


[('DATA_CNT_EV_6M', 0.15587122673199488), ('V_OUT_INT_DUR_3M', nan), ('SMS_CNT_SCH_MORN_6M', nan), ('SUPPORT_3G', 0.094039448280466148), ('DATA_CNT_NGHT_6M', 0.11843377715254405)]
AdaBoost_AUC: 0.721991795242  learning_rate: 0.1  n_est: 300 




In [130]:
from sklearn.feature_selection import RFECV
x_train, y_train = shuffle(data_train[features1], data_train['IS_RETIREE'], random_state=rng)
estimators = [XGBClassifier()]
names = ['ada','ridge','grad','xgb']
for estimator in estimators:
    print(names[estimators.index(estimator)])
    #X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
    selector = RFECV(estimator, step=1, cv=10, scoring = 'roc_auc')
    selector = selector.fit(x_train, y_train)
    print(selector.ranking_)

    features_fin = list(np.array(features1)[selector.support_])
    print([f for f in features1 if f not in features_fin])

ada
[1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 4 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
['OUT_ROUM_MIN_3M', 'OUT_ROUM_CNT_3M', 'IN_ROUM_CNT_3M']


In [ ]:
x_train, y_train = shuffle(data_train[features1], data_train['IS_RETIREE'], random_state=rng)
selector = RFECV(XGBClassifier(), step=1, cv=10, scoring = 'roc_auc')
selector = selector.fit(x_train, y_train)
print(selector.ranking_)

features_fin = list(np.array(features1)[selector.support_])
print([f for f in features1 if f not in features_fin])

### search for best parameters for XGBClassifier and GradientBoostingClassifier

array([[ 0.97244536,  0.02755464],
       [ 0.93554454,  0.06445546],
       [ 0.94251143,  0.05748857],
       ..., 
       [ 0.91301853,  0.08698147],
       [ 0.97319598,  0.02680402],
       [ 0.93252713,  0.06747287]])

In [138]:
#for col in [i for i in features if i not in ['IS_RETIREE','SUPPORT_3G']]:
#    data_train[col] = data_train[col].fillna(0)
#data_kmeans = choose_k_best(38, data_train, features)   
X, y = shuffle(data_train_with_na[features1], data_train_with_na['IS_RETIREE'], random_state=rng)

skf = cv.StratifiedKFold(y, n_folds=10, shuffle=True)
score_metric = 'roc_auc'
scores = {}

def score_model(model):
    return cv.cross_val_score(model, X, y, cv=skf, scoring=score_metric)

dict_xgb = {
        "gamma":[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
        "missing" : [np.nan,],
        "max_depth": [10,9,8,7,6,5,4,3,],
        "colsample_bylevel":[1,0.9,0.8,0.7,0.6,0.5,],
        "n_estimators": [460, 410, 360, 310, 260],
        "learning_rate": [0.5,0.45,0.4,0.3,0.2,0.1,0.05,0.01,0.001],
        "nthread":[30,20,18,16,14,12,10,8,6,5,4,3],
        "subsample":[1,0.9, 0.85, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3],
        "colsample_bytree":[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,],
        "min_child_weight":[1,0.9,0.8,0.7,0.6,0.5],
        "seed":[40,41,50,100,2022,4441]}

params_list = ['max_depth','subsample','colsample_bylevel',
               'n_estimators','learning_rate','nthread','colsample_bytree','gamma','min_child_weight']
cur_params = dict(zip(params_list, [2, 0.95, 0.9, 360, 0.01, 9, 0.3, 0, 1]))
best_params = dict(zip(params_list, [2, 0.95, 0.9, 360, 0.01, 9, 0.3, 0, 1]))

for param in params_list:
    best_AUC = 0
    cur_params = copy.deepcopy(best_params)
    for param_val in dict_xgb[param]:
        cur_params[param] = param_val
        AUC = score_model(XGBClassifier(**cur_params)).mean()
        if AUC > best_AUC + 0.001:
            best_params[param] = param_val
            best_AUC = AUC
            
    print( "Best_AUC:",best_AUC,
        " Parameters:",best_params
         )


Best_AUC: 0.756995713177  Parameters: {'nthread': 9, 'max_depth': 7, 'subsample': 0.95, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.9, 'learning_rate': 0.01, 'min_child_weight': 1, 'n_estimators': 360, 'gamma': 0}
Best_AUC: 0.758911513787  Parameters: {'nthread': 9, 'max_depth': 7, 'subsample': 1, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.9, 'learning_rate': 0.01, 'min_child_weight': 1, 'n_estimators': 360, 'gamma': 0}
Best_AUC: 0.758077893252  Parameters: {'nthread': 9, 'max_depth': 7, 'subsample': 1, 'colsample_bytree': 0.3, 'colsample_bylevel': 1, 'learning_rate': 0.01, 'min_child_weight': 1, 'n_estimators': 360, 'gamma': 0}
Best_AUC: 0.759613835785  Parameters: {'nthread': 9, 'max_depth': 7, 'subsample': 1, 'colsample_bytree': 0.3, 'colsample_bylevel': 1, 'learning_rate': 0.01, 'min_child_weight': 1, 'n_estimators': 460, 'gamma': 0}
Best_AUC: 0.759613835785  Parameters: {'nthread': 9, 'max_depth': 7, 'subsample': 1, 'colsample_bytree': 0.3, 'colsample_bylevel': 1, 'learni

In [140]:
train = data_train.dropna()
print(train.shape)

X, y = shuffle(train_na_dropped[features1], train_na_dropped['IS_RETIREE'], random_state=rng)
skf = cv.StratifiedKFold(y, n_folds=10, shuffle=True)
score_metric = 'roc_auc'
scores = {}

def score_model(model):
    return cv.cross_val_score(model, X, y, cv=skf, scoring=score_metric)

dict_gradboost = {"n_estimators" : [260,360,460,560],
                      "learning_rate": [0.1,0.05,0.2,0.3,0.15,0.35,0.01,0.45,0.5,0.55],
                      "subsample": [0.9,0.8,0.7,0.6,0.5,],
                      "max_leaf_nodes": [8,7,6,5,4,3],
                     "min_samples_split":[8,7,6,5,4,3],
                     "max_features": [7,6,2,4,3,5],
                     "max_depth": [1,2,3],
                     "random_state":[0,100]}

params_list = ['max_depth','n_estimators','learning_rate','subsample','max_leaf_nodes','min_samples_split','max_features']
cur_params = dict(zip(params_list, [2, 500, 0.01, 1, 5, 7, 2]))
best_params2 = dict(zip(params_list, [2, 500, 0.01, 1, 5, 7, 2]))
best_AUC = 0

for param in params_list:
    cur_params = copy.deepcopy(best_params2)
    for param_val in dict_gradboost[param]:
        cur_params[param] = param_val
        AUC = score_model(GradientBoostingClassifier(**cur_params)).mean()
        if AUC > best_AUC + 0.001:
            best_params2[param] = param_val
            best_AUC = AUC
            
            
    print( "Best_AUC:",best_AUC,
        " Parameters:",best_params2
         )



(8532, 56)
Best_AUC: 0.751393527764  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample': 1, 'max_features': 2, 'learning_rate': 0.01, 'n_estimators': 500, 'max_leaf_nodes': 5}
Best_AUC: 0.751393527764  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample': 1, 'max_features': 2, 'learning_rate': 0.01, 'n_estimators': 500, 'max_leaf_nodes': 5}
Best_AUC: 0.758033363674  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample': 1, 'max_features': 2, 'learning_rate': 0.05, 'n_estimators': 500, 'max_leaf_nodes': 5}
Best_AUC: 0.759379861638  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample': 0.8, 'max_features': 2, 'learning_rate': 0.05, 'n_estimators': 500, 'max_leaf_nodes': 5}
Best_AUC: 0.759379861638  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample': 0.8, 'max_features': 2, 'learning_rate': 0.05, 'n_estimators': 500, 'max_leaf_nodes': 5}
Best_AUC: 0.759379861638  Parameters: {'min_samples_split': 7, 'max_depth': 3, 'subsample'

In [141]:
def choose_k_best(k,dataset,features):
    selection = SelectKBest(k=k)
    return SelectKBest(k=k).fit_transform(dataset[features],dataset['IS_RETIREE'])

def fin_Check_GradBoost(features, dataset, k_list):  
    def score_model(model):
        return cv.cross_val_score(model, X, y, cv=skf, scoring=score_metric)
    AUC_grad = list()
    AUC_xgb = list()
    for k in k_list:
        data_kbest = choose_k_best(k, dataset, features)   
        X, y = shuffle(data_kbest, dataset['IS_RETIREE'], random_state=rng)
        
        skf = cv.StratifiedKFold(y, n_folds=10, shuffle=True)
        score_metric = 'roc_auc'
        scores = {}
        
        scores['grad_boost'] = score_model(GradientBoostingClassifier(**best_params2))
        scores['xgb'] = score_model(XGBClassifier(**best_params))
        # Print the scores
        model_scores = pd.DataFrame(scores).mean()
        print("Number of best features chosen:",k)
        print('Model scores average\n{}'.format(model_scores))
        AUC_grad.append(model_scores['grad_boost'])
        AUC_xgb.append(model_scores['xgb'])
    return (AUC_grad, AUC_xgb)   
print("\nWithout dummies, NAs filled with 0:")
k_list = list(range(12, len(features1)+1))
AUC_grad, AUC_xgb = fin_Check_GradBoost(features1, data_train, k_list)
plt.plot(k_list, AUC_grad)
plt.plot(k_list, AUC_xgb)
plt.show()


Without dummies, NAs filled with 0:
Number of best features chosen: 12
Model scores average
grad_boost    0.726958
xgb           0.715972
dtype: float64
Number of best features chosen: 13
Model scores average
grad_boost    0.728080
xgb           0.714606
dtype: float64
Number of best features chosen: 14
Model scores average
grad_boost    0.741615
xgb           0.733443
dtype: float64
Number of best features chosen: 15
Model scores average
grad_boost    0.739566
xgb           0.735990
dtype: float64
Number of best features chosen: 16
Model scores average
grad_boost    0.741532
xgb           0.735403
dtype: float64
Number of best features chosen: 17
Model scores average
grad_boost    0.741249
xgb           0.739804
dtype: float64
Number of best features chosen: 18
Model scores average
grad_boost    0.740369
xgb           0.736807
dtype: float64
Number of best features chosen: 19
Model scores average
grad_boost    0.741497
xgb           0.732624
dtype: float64
Number of best features cho

## Load test dataset and prepare for predictions
## (tranformation is the same as with training dataset)

In [142]:
data_test = pd.read_csv(r'A:\KAGGLE\Kaggle_Kyivstar\Xtest.csv', sep = ',')

category = list(set(data_test['HANDSET_VENDOR_CVAL']))
category
data_test['HANDSET_VENDOR_CVAL_ix'] = [category.index(i) for i in data_test['HANDSET_VENDOR_CVAL']]
category = list(set(data_test['HANDSET_TYPE']))
category
data_test['HANDSET_TYPE_ix'] = [category.index(i) for i in data_test['HANDSET_TYPE']]
data_train['AVG_CHARGE_3M_GROUP'] = data_train['AVG_CHARGE_3M_GROUP']-1
data_test['SUPPORT_3G'] = data_test['SUPPORT_3G'].fillna(0)
data_test = pd.concat([data_test, pd.get_dummies(data_test['AVG_CHARGE_3M_GROUP'],prefix='gr')], axis=1)
data_test = pd.concat([data_test, pd.get_dummies(data_test['HANDSET_TYPE_ix'],prefix='handset')], axis=1)
for feature in features1:
    data_test[feature] = data_test[feature].fillna(0)
#print(data_test.count())


In [113]:
## Make predictions:

x_train, y_train = shuffle(data_train[features_fin], data_train['IS_RETIREE'], random_state=0)
params_list_xgb = ['max_depth','subsample','colsample_bylevel',
               'n_estimators','learning_rate','nthread','colsample_bytree','gamma','min_child_weight']
params_xgb = dict(zip(params_list_xgb, [2, 1, 0.9, 400, 0.01, 9, 0.5, 0, 1]))
fin_model = GradientBoostingClassifier(**best_params).fit(x_train, y_train)
#print(list(zip(features_fin, fin_model.feature_importances_)))
predictions = fin_model.predict_proba(data_test[features_fin])
data_test['IS_RETIREE'] = predictions[:,1]
print(predictions.mean(),predictions.min(),predictions.max())
print(sum(predictions<0.1))
print(sum(predictions>0.9))

0.5 0.00309129282327 0.996908707177
[   0 3524]
[3524    0]


In [112]:

import csv
data_test.to_csv('A:\KAGGLE\Kaggle_Kyivstar\Prediction.csv', 
                                              index= False, columns = ['SUBS_ID', 'IS_RETIREE'],quoting=csv.QUOTE_NONNUMERIC)